<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/udemy_pyomo/pyomo_ip_tut2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 2

A firm thar produces automobile seats manufactures 3 seat types on 2 different production lines.  
Up to 30 workers can be used at the same time on each production line to produce any seat type.  
Each worker is paid \$400 per week on production line 1, and \$600 per week on production line 2.   
1 week of production on line 1 cost \$1000, and cost \$2000 on line 2.  
Below table shows the number of seats by each worker on a production line.  

|Line|Seat1|Seat2|seat3|  
|----|-----|-----|-----|  
|  1 | 20  |  30 |  40 |  
|  2 | 50  |  35 |  45 |   

The weekly demand of seat:  
seat1: 120.  
seat2: 150.  
seat3: 200.  

Use Integer Linear Programming to minimize the total cost of production plan.  




## Mathematical model

Let the demand of each seat type be D_i.  
Let the number of each type of seat produced on each line be X_i,j.  
Let the cost of each worker be W_j.  
Let the cost of each line be L_j.  
Let the decsion of starting a line be S_j.  
Let the big M for each line be M_j.  
Let the number of each worker on each line be N_j.   
Let i be in the range 1 to 3.   
Let j be in the range 1 to 2.  

$$  
\\objective~ function:  
\\min(\sum_{j=1}(N_j \times W_j) + \sum_{j=1}(S_j\times L_j))
\\s.t  
\\\sum(X_{i,j}\times N_j)\geq D_i
\\N_j \leq S_j\times M_j  
$$  

In [1]:
# Install denpendencies
!pip install pyomo
!apt-get install -y -qq glpk-utils

In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [3]:
# Define the indicies
i = ['Seat1', 'Seat2', 'Seat3']
j = ['Line1', 'Line2']

# Define the Parameters
D = {'Seat1':120, 'Seat2':150, 'Seat3':200}
W = {'Line1':400, 'Line2':600}
L = {'Line1':1000, 'Line2':2000}
X = {('Seat1', 'Line1'):20, ('Seat2', 'Line1'):30, ('Seat3', 'Line1'):40,
     ('Seat1', 'Line2'):50, ('Seat2', 'Line2'):35, ('Seat3', 'Line2'):45}





In [4]:
# Create model
model = pyo.ConcreteModel()

# Set pyomo set as indicies.
model.i = pyo.Set(initialize=i)
model.j = pyo.Set(initialize=j)

# Set pyomo parameters
model.Demand = pyo.Param(model.i, initialize=D)
demand = model.Demand
model.Worker_salary = pyo.Param(model.j, initialize=W)
worker_sal = model.Worker_salary
model.line_cost = pyo.Param(model.j, initialize=L)
line_cost = model.line_cost
model.production_eff = pyo.Param(model.i, model.j, initialize=X)
production_eff = model.production_eff

In [5]:
big_M = 30
# Create the decision variables
model.num_workers = pyo.Var(model.j, within=pyo.NonNegativeIntegers)
num_workers= model.num_workers
model.num_lines = pyo.Var(model.j, within=pyo.Binary)
num_lines = model.num_lines

# Define the objective function
def min_cost(model):
  worker_cost = sum(model.Worker_salary[j] * model.num_workers[j]for j in model.j)
  line_cost = sum(model.num_lines[j] * model.line_cost[j] for j in model.j)
  total_cost = worker_cost + line_cost
  return total_cost
model.Objf = pyo.Objective(rule = min_cost, sense = pyo.minimize)

In [6]:
# Define the constraints
def demand_const(model, i):
  return sum(model.num_workers[j]*model.production_eff[i,j] for j in model.j) >= model.Demand[i]

model.demand_constraint = pyo.ConstraintList()
for i in model.i:
  model.demand_constraint.add(
      expr=demand_const(model, i)
  )

def big_M(model, j):
  return model.num_workers[j] <= 30 * model.num_lines[j]
model.big_M = pyo.ConstraintList()
for j in model.j:
  model.big_M.add(
      expr=big_M(model, j)
  )

In [7]:
# Create solver factory
Solver = SolverFactory('glpk')
results =Solver.solve(model)


In [8]:
print(results)
print('objective function: ', model.Objf())


Problem: 
- Name: unknown
  Lower bound: 3400.0
  Upper bound: 3400.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.011162042617797852
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

objective function:  3400.0


In [10]:
for j in model.j:
  print(f"The number of worker on {j} is {model.num_workers[j]()}")

The number of worker on Line1 is 6.0
The number of worker on Line2 is 0.0
